## ⚠️ BEFORE YOU START

**What day is it today?**

- ✅ **Friday after 4pm ET:** Perfect! Run all cells below
- ❌ **Monday-Thursday:** STOP! Wait until Friday after market close
- ❌ **Friday before 4pm:** Wait until market closes at 4pm ET

**Why Friday close?** GHB Strategy uses weekly closing prices. Trading mid-week gives false signals and won't match backtested performance.

---

# GHB Strategy Portfolio Scanner
**Gold-Gray-Blue Weekly Trading System**

## 📅 WEEKLY TRADING SCHEDULE

⚠️ **IMPORTANT:** Only run this scanner on **FRIDAY after 4pm ET** (after market close)

### Your Weekly Routine:

**FRIDAY (After 4pm ET)**
- 📊 Run this notebook (all cells)
- 📋 Review signals: BUY (P1), HOLD (P2/N1), SELL (N2)
- 📝 Make your trade list for Monday
- ⏱️ Time: 10-15 minutes

**WEEKEND (Saturday/Sunday)**
- 💭 Review and confirm your plan
- 🧮 Calculate position sizes (10% each = $11,000 per position)
- ✅ Prepare for Monday execution

**MONDAY (Market Open - 9:30am ET)**
- 🔵 **FIRST (9:30-10:00am):** Execute ALL sell signals (N2 stocks) - URGENT
- 🟡 **THEN (10:00-10:30am):** Enter new buy positions (P1 stocks) - PATIENT
- ⏱️ Time: 15-30 minutes
- 💡 Tip: Use limit orders (Sells: Friday -1%, Buys: Friday +1.5%)

---

**Last Run:** {current_date}  
**Strategy:** GHB Strategy (Gold-Gray-Blue)  
**Universe:** 25 S&P 500 Optimized Stocks (Top-ranked by backtest CAGR)  
**Configuration:** 10% position size, 10 max holdings  
**Expected Performance:** 46.80% CAGR | 62.86% Win Rate (2021-2025 backtest)

In [ ]:
# Import Required Libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")
print(f"📅 Current Date: {datetime.now().strftime('%Y-%m-%d')}")

## 1. Define Stock Universe

The 25 stocks selected for optimal performance:
- Your 12-stock watchlist (core holdings)
- 13 top performers from backtesting (2021-2025)
- Expected: +514% annual returns, ~14 trades/year

This delivers 117% of full universe returns with much easier management!

In [ ]:
# GHB Strategy S&P 500 Optimized Portfolio - 25 Stocks
# Top-ranked S&P 500 stocks meeting GHB volatility criteria (2021-2025 backtest)
GHB_UNIVERSE = [
    'ANET', 'APH', 'AXON', 'AVGO', 'CAH', 'CEG', 'DECK', 'DVN', 'GE',
    'GOOG', 'GOOGL', 'JPM', 'LLY', 'MCK', 'MPC', 'MU', 'NFLX', 'NVDA',
    'ORCL', 'PWR', 'SMCI', 'STX', 'TRGP', 'VST', 'WMB'
]

print(f"📊 Universe: {len(GHB_UNIVERSE)} stocks (S&P 500 Optimized)")
print(f"📈 Stocks: {', '.join(sorted(GHB_UNIVERSE[:10]))}...")
print(f"💡 Expected: 46.80% CAGR, ~7 trades/year, 62.86% win rate")

## 2. Calculate Weekly Larsson States

For each stock, calculate:
- **Weekly Close** (Friday)
- **200-Day SMA** (D200)
- **4-Week ROC** (Rate of Change)
- **Weekly State** (P1/P2/N1/N2)

In [ ]:
def calculate_weekly_larsson_state(ticker):
    """
    Calculate weekly Larsson state for a ticker
    Returns: dict with ticker data or None if error
    """
    try:
        # Download 1 year of daily data
        stock = yf.Ticker(ticker)
        df = stock.history(period='1y', interval='1d')
        
        if df.empty or len(df) < 200:
            return None
        
        # Get latest close
        close = df['Close'].iloc[-1]
        
        # Calculate 200-day SMA
        d200 = df['Close'].rolling(window=200).mean().iloc[-1]
        
        # Calculate 4-week ROC (20 trading days)
        if len(df) >= 20:
            price_4w_ago = df['Close'].iloc[-20]
            roc_4w = ((close - price_4w_ago) / price_4w_ago) * 100
        else:
            roc_4w = 0
        
        # Calculate distance from D200
        distance_pct = ((close - d200) / d200) * 100
        
        # Determine Weekly Larsson state (Strategy D rules)
        if close > d200:
            # Price above D200
            if roc_4w > 5 or distance_pct > 10:
                state = 'P1'  # Strong bullish
                signal = '🟡 BUY'
            else:
                state = 'P2'  # Consolidation
                signal = '⚪ HOLD'
        else:
            # Price below D200
            if distance_pct > -5:
                state = 'N1'  # Shallow pullback
                signal = '⚪ HOLD'
            else:
                state = 'N2'  # Downtrend
                signal = '🔵 SELL'
        
        return {
            'Ticker': ticker,
            'Close': close,
            'D200': d200,
            'Distance_%': distance_pct,
            'ROC_4W_%': roc_4w,
            'State': state,
            'Signal': signal
        }
        
    except Exception as e:
        print(f"❌ Error processing {ticker}: {str(e)}")
        return None

print("✅ Calculation function defined")

## 3. Scan All 39 Stocks

This will take 1-2 minutes to download data and calculate states...

In [ ]:
print(f"🔄 Scanning {len(GHB_UNIVERSE)} stocks... Please wait...\n")

results = []
for i, ticker in enumerate(GHB_UNIVERSE, 1):
    print(f"  [{i:2d}/{len(GHB_UNIVERSE)}] Processing {ticker:6s}...", end='\r')
    result = calculate_weekly_larsson_state(ticker)
    if result:
        results.append(result)

df_results = pd.DataFrame(results)

print(f"\n✅ Scan complete! Processed {len(df_results)}/{len(GHB_UNIVERSE)} stocks")
print(f"❌ Failed: {len(GHB_UNIVERSE) - len(df_results)} stocks")

## 4. Strategy D Signals

### Buy Signals (P1 - Gold)
**Action:** Enter new positions or add to existing  
**Requirements:** Price > D200 + Strong momentum

In [ ]:
# Filter P1 (BUY) signals
p1_signals = df_results[df_results['State'] == 'P1'].sort_values('ROC_4W_%', ascending=False)

print("=" * 100)
print(f"🟡 P1 (GOLD) - BUY SIGNALS: {len(p1_signals)} stocks")
print("=" * 100)

if len(p1_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Above D200':<12} {'4W ROC':<10} {'Strength':<15}")
    print("-" * 100)
    
    for _, row in p1_signals.iterrows():
        # Determine strength
        if row['Distance_%'] > 30:
            strength = "🔥 EXPLOSIVE"
        elif row['Distance_%'] > 20:
            strength = "💪 VERY STRONG"
        elif row['ROC_4W_%'] < 0:
            strength = "✅ PULLBACK BUY"
        else:
            strength = "✅ STRONG"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {strength:<15}")
    
    print("\n💡 RECOMMENDATION:")
    print("   - Enter NEW positions in these P1 stocks")
    print("   - Add to existing positions if capital available")
    print("   - Prioritize: Explosive > Very Strong > Strong > Pullback Buy")
else:
    print("\n⚠️  No P1 buy signals this week")
    print("   - Wait for new P1 entries")
    print("   - Hold existing positions")

### Hold Signals (P2 & N1 - Gray)
**Action:** Continue holding existing positions  
**Meaning:** Normal consolidation/pullback in trend

In [ ]:
# Filter P2 and N1 (HOLD) signals
hold_signals = df_results[df_results['State'].isin(['P2', 'N1'])].sort_values('Distance_%', ascending=False)

print("=" * 100)
print(f"⚪ P2/N1 (GRAY) - HOLD SIGNALS: {len(hold_signals)} stocks")
print("=" * 100)

if len(hold_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Distance':<12} {'4W ROC':<10} {'State':<8} {'Status':<20}")
    print("-" * 100)
    
    for _, row in hold_signals.iterrows():
        if row['State'] == 'P2':
            status = "Consolidation"
        else:
            status = "Shallow Pullback"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {row['State']:<8} {status:<20}")
    
    print("\n💡 RECOMMENDATION:")
    print("   - HOLD all existing positions")
    print("   - Do NOT sell - this is normal consolidation")
    print("   - Watch for transition to P1 (upgrade) or N2 (downgrade)")
else:
    print("\n✅ No stocks in consolidation phase")

### Sell Signals (N2 - Blue)
**Action:** Exit positions immediately  
**Requirements:** Price < D200 + Weak momentum  
**⚠️ CRITICAL: Execute these sells on Monday!**

In [ ]:
# Filter N2 (SELL) signals
n2_signals = df_results[df_results['State'] == 'N2'].sort_values('Distance_%', ascending=True)

print("=" * 100)
print(f"🔵 N2 (BLUE) - SELL SIGNALS: {len(n2_signals)} stocks")
print("=" * 100)

if len(n2_signals) > 0:
    print(f"\n⚠️  EXIT THESE POSITIONS ON MONDAY!\n")
    print(f"{'Ticker':<8} {'Price':<10} {'D200':<10} {'Below D200':<12} {'4W ROC':<10} {'Severity':<15}")
    print("-" * 100)
    
    for _, row in n2_signals.iterrows():
        # Determine severity
        if row['Distance_%'] < -20:
            severity = "🚨 SEVERE"
        elif row['Distance_%'] < -10:
            severity = "⚠️  MAJOR"
        else:
            severity = "📉 MINOR"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {severity:<15}")
    
    print("\n💡 ACTION REQUIRED:")
    print("   - SELL all N2 positions on Monday at market open")
    print("   - Do NOT wait for bounce - trend is broken")
    print("   - Preserve capital for new P1 opportunities")
else:
    print("\n✅ No sell signals - all positions healthy!")

## 5. Weekly Summary

Quick overview of portfolio status and action items

In [ ]:
print("=" * 100)
print("📊 GHB STRATEGY WEEKLY SUMMARY")
print("=" * 100)

print(f"\n🟡 BUY Signals (P1):  {len(p1_signals)} stocks")
print(f"⚪ HOLD Signals (P2/N1): {len(hold_signals)} stocks")
print(f"🔵 SELL Signals (N2): {len(n2_signals)} stocks")
print(f"📊 Total Scanned: {len(df_results)}/{len(GHB_UNIVERSE)} stocks")

# Calculate portfolio health
total_bullish = len(p1_signals)
total_neutral = len(hold_signals)
total_bearish = len(n2_signals)
total = len(df_results)

pct_bullish = (total_bullish / total * 100) if total > 0 else 0
pct_bearish = (total_bearish / total * 100) if total > 0 else 0

print(f"\n📈 Market Health:")
print(f"   Bullish: {pct_bullish:.1f}% ({total_bullish} stocks)")
print(f"   Neutral: {(total_neutral/total*100):.1f}% ({total_neutral} stocks)")
print(f"   Bearish: {pct_bearish:.1f}% ({total_bearish} stocks)")

if pct_bullish > 60:
    market_sentiment = "🟢 VERY BULLISH - Many opportunities"
elif pct_bullish > 40:
    market_sentiment = "🟡 BULLISH - Good opportunities"
elif pct_bullish > 20:
    market_sentiment = "🟠 NEUTRAL - Selective opportunities"
else:
    market_sentiment = "🔴 BEARISH - Few opportunities, preserve cash"

print(f"\n📊 Market Sentiment: {market_sentiment}")

# Action items
print("\n✅ ACTION ITEMS FOR THIS WEEK:")
if len(n2_signals) > 0:
    print(f"   1. MONDAY: Sell {len(n2_signals)} N2 positions at market open")
else:
    print("   1. No sells required")

if len(p1_signals) > 0:
    print(f"   2. MONDAY: Enter up to {min(5, len(p1_signals))} new P1 positions")
    print(f"      → Priority: {', '.join(p1_signals.head(5)['Ticker'].tolist())}")
else:
    print("   2. No new buys available - hold cash")

if len(hold_signals) > 0:
    print(f"   3. Monitor {len(hold_signals)} holding positions for state changes")
else:
    print("   3. No positions to monitor")

print("\n" + "=" * 100)

## 6. Detailed Stock Data

Full dataset for analysis and record-keeping

In [ ]:
# Display full results sorted by state then distance
df_display = df_results.copy()
df_display['State_Order'] = df_display['State'].map({'P1': 1, 'P2': 2, 'N1': 3, 'N2': 4})
df_display = df_display.sort_values(['State_Order', 'Distance_%'], ascending=[True, False])
df_display = df_display.drop('State_Order', axis=1)

print("\n📋 COMPLETE SCAN RESULTS")
print("=" * 100)
print(df_display.to_string(index=False))
print("=" * 100)

## 6.5 Current Portfolio Holdings

**Phase 1 - Portfolio Tracker:** Track your positions and calculate performance

In [ ]:
import json
from pathlib import Path

# Load portfolio settings
settings_path = Path("../data/portfolio_settings.json")
with open(settings_path, 'r') as f:
    portfolio_settings = json.load(f)

starting_cash = portfolio_settings['starting_cash']
position_size_pct = portfolio_settings['position_size_pct']
max_positions = portfolio_settings['max_positions']
strategy_week = portfolio_settings['strategy_week']
conservative_mode = portfolio_settings.get('conservative_mode', True)

print("💼 PORTFOLIO CONFIGURATION")
print("=" * 100)
print(f"Starting Capital: ${starting_cash:,.0f}")
print(f"Position Size: {position_size_pct}% (${starting_cash * position_size_pct / 100:,.0f} per position)")
print(f"Max Positions: {max_positions}")
print(f"Strategy Week: {strategy_week}")
print(f"Mode: {'Conservative (Building Gradually)' if conservative_mode else 'Aggressive'}")
print("=" * 100)

# Load current positions
positions_path = Path("../data/portfolio_positions.csv")
df_positions = pd.read_csv(positions_path)

# Initialize variables for both cases
position_summaries = []
total_cost = 0
total_value = 0
total_pl = 0
total_pl_pct = 0
cash_remaining = starting_cash
deployed_pct = 0
position_value = starting_cash * position_size_pct / 100

if len(df_positions) == 0:
    print("\n📭 No positions yet - Portfolio is 100% CASH")
    print(f"💰 Available: ${starting_cash:,.0f}")
    print(f"\n💡 Week {strategy_week} Recommendation:")
    if strategy_week == 1:
        print("   Start with 3 positions (21% deployed)")
        print(f"   Keep ${starting_cash * 0.79:,.0f} in cash for future opportunities")
    else:
        print(f"   Consider adding 2-3 new positions")
        print(f"   Build towards {max_positions} total positions")
else:
    print(f"\n📊 ACTIVE POSITIONS: {len(df_positions)}")
    print("=" * 100)
    
    # Update current states for all positions
    position_summaries = []
    
    for _, pos in df_positions.iterrows():
        ticker = pos['Ticker']
        
        # Get current data from scan results
        current_data = df_results[df_results['Ticker'] == ticker]
        
        if len(current_data) > 0:
            current_price = current_data.iloc[0]['Close']
            current_state = current_data.iloc[0]['State']
            current_signal = current_data.iloc[0]['Signal']
            distance_pct = current_data.iloc[0]['Distance_%']
            roc_pct = current_data.iloc[0]['ROC_4W_%']
        else:
            # Ticker not in universe or failed to scan
            current_price = pos['Entry_Price']  # Fallback
            current_state = pos['Current_State']
            current_signal = '❓ UNKNOWN'
            distance_pct = 0
            roc_pct = 0
        
        # Calculate P&L
        entry_price = pos['Entry_Price']
        shares = pos['Shares']
        cost_basis = entry_price * shares
        current_value = current_price * shares
        pl_dollars = current_value - cost_basis
        pl_pct = ((current_price - entry_price) / entry_price) * 100
        
        # State change detection
        state_change = ""
        if current_state != pos['Entry_State']:
            state_change = f"({pos['Entry_State']} → {current_state})"
        
        position_summaries.append({
            'Ticker': ticker,
            'Entry_Date': pos['Entry_Date'],
            'Entry_Price': f"${entry_price:.2f}",
            'Current_Price': f"${current_price:.2f}",
            'Shares': int(shares),
            'Cost_Basis': f"${cost_basis:,.0f}",
            'Current_Value': f"${current_value:,.0f}",
            'P/L_$': f"${pl_dollars:+,.0f}",
            'P/L_%': f"{pl_pct:+.1f}%",
            'Entry_State': pos['Entry_State'],
            'Current_State': current_state,
            'State_Change': state_change,
            'Signal': current_signal
        })
    
    df_summary = pd.DataFrame(position_summaries)
    print(df_summary.to_string(index=False))
    print("=" * 100)
    
    # Portfolio totals
    total_cost = df_positions['Entry_Price'].sum() * df_positions['Shares'].sum() if len(df_positions) > 0 else 0
    total_value = sum([float(s['Current_Value'].replace('$', '').replace(',', '')) for s in position_summaries])
    total_pl = total_value - total_cost
    total_pl_pct = (total_pl / total_cost * 100) if total_cost > 0 else 0
    
    cash_remaining = starting_cash - total_cost
    deployed_pct = (total_cost / starting_cash * 100)
    
    print(f"\n💼 PORTFOLIO SUMMARY")
    print("=" * 100)
    print(f"Total Cost Basis: ${total_cost:,.0f}")
    print(f"Current Value: ${total_value:,.0f}")
    print(f"Total P/L: ${total_pl:+,.0f} ({total_pl_pct:+.1f}%)")
    print(f"Cash Remaining: ${cash_remaining:,.0f}")
    print(f"Deployed: {deployed_pct:.1f}% | Cash: {100-deployed_pct:.1f}%")
    print("=" * 100)
    
    # Alerts and Warnings
    print(f"\n⚠️ PORTFOLIO ALERTS")
    print("=" * 100)
    
    n2_positions = [s for s in position_summaries if 'SELL' in s['Signal']]
    state_changes = [s for s in position_summaries if s['State_Change'] != ""]
    
    if len(n2_positions) > 0:
        print(f"🔴 URGENT: {len(n2_positions)} position(s) in N2 (SELL) state!")
        for pos in n2_positions:
            print(f"   → {pos['Ticker']}: SELL on Monday open (Current: {pos['Current_State']})")
    else:
        print("✅ No urgent sell signals")
    
    if len(state_changes) > 0:
        print(f"\n📊 {len(state_changes)} position(s) changed state:")
        for pos in state_changes:
            print(f"   → {pos['Ticker']}: {pos['State_Change']}")
    else:
        print("\n✅ All positions maintained their states")
    
    print("=" * 100)
    
    # Position sizing recommendations
    print(f"\n💰 POSITION SIZING FOR THIS WEEK")
    print("=" * 100)
    
    recommended_adds = max_positions - len(df_positions)
    position_value = starting_cash * position_size_pct / 100
    
    if recommended_adds > 0 and len(p1_signals) > 0:
        print(f"📈 Recommended: Add {min(recommended_adds, len(p1_signals))} new position(s)")
        print(f"💵 Per Position: ${position_value:,.0f} ({position_size_pct}% of starting capital)")
        print(f"🎯 Total Deploy: ${position_value * min(recommended_adds, len(p1_signals)):,.0f}")
        print(f"📊 New Deployment: {(total_cost + position_value * min(recommended_adds, len(p1_signals))) / starting_cash * 100:.1f}%")
        print(f"\n💡 Top {min(recommended_adds, 3)} Candidates:")
        for i, ticker in enumerate(p1_signals.head(min(recommended_adds, 3))['Ticker']):
            print(f"   {i+1}. {ticker} - Enter ${position_value:,.0f} position")
    elif len(df_positions) >= max_positions:
        print(f"✅ Portfolio full ({len(df_positions)}/{max_positions} positions)")
        print(f"💡 Only trade if N2 sell creates opening")
    else:
        print(f"⚠️ No P1 buy signals available this week")
        print(f"💼 Hold {len(df_positions)} positions and ${cash_remaining:,.0f} cash")
    
    print("=" * 100)

## 6.6 How to Add New Positions

**MANUAL WORKFLOW (Phase 1):**

1. **Friday:** Run this notebook to get signals
2. **Monday:** Execute trades (SELL first, then BUY)
3. **Monday Evening:** Manually update `data/portfolio_positions.csv`

**Example Entry:**
```
TSLA,2026-01-20,450.00,17,P1,P1,🟡 BUY
```

**Fields:**
- `Ticker`: Stock symbol
- `Entry_Date`: YYYY-MM-DD format
- `Entry_Price`: Your fill price
- `Shares`: Number of shares purchased
- `Entry_State`: P1 (the state when you bought)
- `Current_State`: P1 (will auto-update next week)
- `Entry_Signal`: 🟡 BUY

Next Friday, the scanner will automatically update Current_State and calculate P/L!

In [ ]:
"""
Universe Health Check & Re-Optimization Alerts
Automatically flag conditions that suggest universe needs refresh
"""

from datetime import datetime, timedelta

print("\n" + "="*80)
print("🔍 UNIVERSE HEALTH CHECK - Re-Optimization Alerts")
print("="*80)

# Initialize alert tracking
reopt_alerts = []
alert_severity = "GREEN"  # GREEN, YELLOW, RED

# --- CONDITION 1: Universe Degradation (>30% in N2) ---
n2_count = len(n2_signals)
total_universe = len(df_results)
n2_percentage = (n2_count / total_universe * 100) if total_universe > 0 else 0

print(f"\n📊 Condition 1: Universe Degradation")
print(f"   N2 Stocks: {n2_count}/{total_universe} ({n2_percentage:.1f}%)")

if n2_percentage > 30:
    severity = "🔴 CRITICAL"
    alert_severity = "RED"
    reopt_alerts.append({
        'condition': 'Universe Degradation',
        'severity': 'CRITICAL',
        'detail': f'{n2_percentage:.1f}% of universe in N2 (threshold: 30%)',
        'action': 'Re-screen S&P 500 immediately - universe is broken'
    })
    print(f"   {severity}: {n2_percentage:.1f}% in N2 (Threshold: 30%)")
    print(f"   ⚠️  ACTION: Re-screen S&P 500 NOW - universe showing widespread weakness")
elif n2_percentage > 20:
    severity = "🟡 WARNING"
    if alert_severity == "GREEN":
        alert_severity = "YELLOW"
    reopt_alerts.append({
        'condition': 'Universe Degradation',
        'severity': 'WARNING',
        'detail': f'{n2_percentage:.1f}% of universe in N2 (watch threshold: 20%)',
        'action': 'Monitor closely - consider re-screening if persists 2+ weeks'
    })
    print(f"   {severity}: {n2_percentage:.1f}% in N2 (Watch at 20%)")
    print(f"   💡 Monitor: If this persists for 2+ weeks, consider re-screening")
else:
    print(f"   ✅ HEALTHY: {n2_percentage:.1f}% in N2 (Normal: <20%)")

# --- CONDITION 2: Performance Tracking (if portfolio exists) ---
print(f"\n📈 Condition 2: Portfolio Performance vs Expected")

if len(df_positions) > 0:
    # Calculate expected return based on time held
    # Expected: 46.80% annual = 0.90% per week
    expected_weekly_return = 0.90  # 46.80% / 52 weeks
    
    # Calculate actual return
    total_pl_pct_calc = ((total_value - total_cost) / total_cost * 100) if total_cost > 0 else 0
    
    # Estimate weeks held (simplified - assumes all positions entered at same time)
    weeks_held = strategy_week  # Using strategy week as proxy
    expected_return = expected_weekly_return * weeks_held
    
    performance_gap = total_pl_pct_calc - expected_return
    
    print(f"   Actual Return: {total_pl_pct_calc:+.1f}%")
    print(f"   Expected Return ({weeks_held} weeks): {expected_return:+.1f}%")
    print(f"   Performance Gap: {performance_gap:+.1f}%")
    
    if performance_gap < -10 and weeks_held > 12:
        severity = "🟡 WARNING"
        if alert_severity == "GREEN":
            alert_severity = "YELLOW"
        reopt_alerts.append({
            'condition': 'Performance Lag',
            'severity': 'WARNING',
            'detail': f'Portfolio underperforming by {abs(performance_gap):.1f}% (threshold: -10%)',
            'action': 'Review stock selection - consider re-screening'
        })
        print(f"   {severity}: Underperforming by {abs(performance_gap):.1f}%")
    else:
        print(f"   ✅ ON TRACK: Performance within expected range")
else:
    print(f"   ⏳ No positions yet - tracking starts after first trades")

# --- CONDITION 3: Low Opportunity Environment ---
print(f"\n🎯 Condition 3: Market Opportunities")

p1_percentage = (len(p1_signals) / total_universe * 100) if total_universe > 0 else 0
print(f"   P1 (BUY) Signals: {len(p1_signals)}/{total_universe} ({p1_percentage:.1f}%)")

if p1_percentage < 20:
    severity = "🟡 WARNING"
    if alert_severity == "GREEN":
        alert_severity = "YELLOW"
    reopt_alerts.append({
        'condition': 'Low Opportunity',
        'severity': 'WARNING',
        'detail': f'Only {p1_percentage:.1f}% showing BUY signals',
        'action': 'Market may be bearish - check if universe needs refresh'
    })
    print(f"   {severity}: Very few opportunities")
else:
    print(f"   ✅ HEALTHY: {p1_percentage:.1f}% in P1")

# --- CONDITION 4: Time Since Last Update ---
print(f"\n📅 Condition 4: Universe Age")

portfolio_file = Path("../data/ghb_optimized_portfolio.txt")
if portfolio_file.exists():
    last_modified = datetime.fromtimestamp(portfolio_file.stat().st_mtime)
    days_since = (datetime.now() - last_modified).days
    months_since = days_since / 30.44
    
    print(f"   Last Updated: {last_modified.strftime('%Y-%m-%d')} ({months_since:.1f} months ago)")
    
    if days_since > 365:
        severity = "🔴 CRITICAL"
        alert_severity = "RED"
        reopt_alerts.append({
            'condition': 'Stale Universe',
            'severity': 'CRITICAL',
            'detail': f'Universe {months_since:.1f} months old',
            'action': 'REQUIRED: Annual re-optimization overdue'
        })
        print(f"   {severity}: Over 1 year old - RE-OPTIMIZE NOW")
    elif days_since > 180:
        severity = "🟡 WARNING"
        if alert_severity == "GREEN":
            alert_severity = "YELLOW"
        print(f"   {severity}: Over 6 months - plan update soon")
    else:
        print(f"   ✅ FRESH: Recently updated")

# --- SUMMARY ---
print(f"\n" + "="*80)
print(f"🚦 OVERALL STATUS: {alert_severity}")
print("="*80)

if alert_severity == "RED":
    print(f"🔴 CRITICAL: Re-optimization REQUIRED ({len(reopt_alerts)} issue(s))")
elif alert_severity == "YELLOW":
    print(f"🟡 WARNING: Re-optimization recommended ({len(reopt_alerts)} warning(s))")
else:
    print(f"✅ HEALTHY: No re-optimization needed")

if len(reopt_alerts) > 0:
    print(f"\n📋 ACTION ITEMS:")
    for i, alert in enumerate(reopt_alerts, 1):
        print(f"   {i}. {alert['condition']}: {alert['action']}")
    print(f"\n🔧 TO RE-OPTIMIZE: Run universe_reoptimization.ipynb")

print(f"\n" + "="*80)

## 6.4 Universe Health Check & Re-Optimization Alerts

**Automated monitoring:** Check if universe needs refresh

## 6.7 Monday Execution Guidelines

### ⏰ TIMING STRATEGY

**9:30-10:00am: SELL N2 Positions (URGENT)**
- Execute ALL sells in first 30 minutes
- N2 = Trend broken, losses compound quickly
- Don't wait for bounce - exit fast
- Limit: Friday close - 1% (aggressive exit)

**10:00-10:30am: BUY P1 Positions (PATIENT)**
- Wait for market to settle after opening volatility
- Avoid wide spreads and overnight gap reactions
- Limit: Friday close + 1.5% (balanced entry)
- Better to miss entry than chase a 3%+ gap

### 📊 LIMIT ORDER STRATEGY

**SELL Limits (Aggressive):**
- Set at Friday close × 0.99 (1% below)
- Example: $100 Friday → Limit $99
- Priority: Get out fast, price less important
- If gaps down to $95, you exit at $95 (good!)
- If opens at $100.50, you exit at $100.50 (fine!)

**BUY Limits (Balanced):**
- Set at Friday close × 1.015 (1.5% above)
- Example: $95.50 Friday → Limit $96.93
- Gives room for normal overnight gaps
- Protects against chasing 3%+ moves
- ~90% fill rate vs 60% at exact Friday close

### 🚫 MID-WEEK TRADING RULES

**If BUY limit doesn't fill Monday:**
- ❌ **DON'T chase Tuesday-Thursday**
- ❌ **DON'T try to "catch the pullback"**
- ✅ **WAIT for next Friday's scan**
- ✅ **Enter different position that fills**

**Why wait?**
- Signals based on FRIDAY weekly close only
- Mid-week entry deviates from backtested strategy
- State might change by next Friday anyway
- 14+ trades/year means missing one is fine

**If SELL signal on Friday:**
- ✅ **MUST execute Monday 9:30am**
- ⚠️ If somehow missed, sell Tuesday morning
- ❌ **NEVER wait until next Friday**
- N2 detection already happened - act immediately

### 🎯 ASYMMETRIC STRATEGY

**Selling = Speed**
- Early execution (9:30am)
- Aggressive limits (-1%)
- No patience needed
- Preserve capital fast

**Buying = Patience**
- Delayed execution (10:00am)
- Flexible limits (+1.5%)
- Skip if too expensive
- Wait for next opportunity

### 💡 KEY PRINCIPLE

**Weekly discipline > Perfect execution**

The +514% backtest assumes reasonable execution at WEEKLY signals. Missing a Monday entry and waiting for next Friday is better than mid-week improvisation.

**In a hurry to stop losses, patient to enter winners.**

## 7. Export Results

Save results to CSV for record-keeping and further analysis

In [ ]:
# Archive old scan results
import os
import shutil
from pathlib import Path
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_CENTER, TA_LEFT

# Create folders if they don't exist
results_dir = Path("../ghb_scanner_results")
archive_dir = results_dir / "archive"
results_dir.mkdir(exist_ok=True)
archive_dir.mkdir(exist_ok=True)

# Move old CSV and PDF files to archive
for old_file in list(results_dir.glob("ghb_strategy_signals_*.csv")) + list(results_dir.glob("ghb_strategy_signals_*.pdf")):
    try:
        shutil.move(str(old_file), str(archive_dir / old_file.name))
        print(f"📦 Archived: {old_file.name}")
    except Exception as e:
        print(f"⚠️ Could not archive {old_file.name}: {e}")

# Save CSV with date and time
datetime_str = datetime.now().strftime('%Y%m%d_%H%M')
output_file = results_dir / f"ghb_strategy_signals_{datetime_str}.csv"
df_results.to_csv(output_file, index=False)

# Generate PDF Report with date and time
pdf_file = results_dir / f"ghb_strategy_signals_{datetime_str}.pdf"
doc = SimpleDocTemplate(str(pdf_file), pagesize=letter, topMargin=0.5*inch, bottomMargin=0.5*inch)
styles = getSampleStyleSheet()
story = []

# Title
title_style = ParagraphStyle('CustomTitle', parent=styles['Heading1'], fontSize=18, textColor=colors.HexColor('#1f4788'), alignment=TA_CENTER, spaceAfter=12)
story.append(Paragraph("GHB Strategy Portfolio Scanner", title_style))
subtitle_style = ParagraphStyle('Subtitle', parent=styles['Normal'], fontSize=10, textColor=colors.HexColor('#666666'), alignment=TA_CENTER, spaceAfter=6)
story.append(Paragraph("Gold-Gray-Blue Weekly Trading System", subtitle_style))
story.append(Paragraph(f"Scan Date: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}", styles['Normal']))
story.append(Spacer(1, 0.2*inch))

# State Abbreviations Key
abbrev_style = ParagraphStyle('AbbrevTitle', parent=styles['Heading3'], fontSize=11, textColor=colors.HexColor('#2c5aa0'), spaceAfter=6)
story.append(Paragraph("State Abbreviations", abbrev_style))
abbrev_data = [
    ['State', 'Meaning', 'Signal'],
    ['P1 (Gold)', 'Positive + Strong: Price > 200-day SMA + momentum', 'BUY'],
    ['P2 (Gray)', 'Positive + Weak: Price > 200-day SMA, low momentum', 'HOLD'],
    ['N1 (Gray)', 'Negative + Strong: Price slightly below 200-day SMA', 'HOLD'],
    ['N2 (Blue)', 'Negative + Weak: Price well below 200-day SMA', 'SELL']
]
abbrev_table = Table(abbrev_data, colWidths=[1*inch, 3.5*inch, 0.8*inch])
abbrev_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#e8f0f8')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 9),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
    ('FONTSIZE', (0, 1), (-1, -1), 8),
    ('TOPPADDING', (0, 1), (-1, -1), 4),
    ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ('BACKGROUND', (0, 1), (0, 1), colors.HexColor('#fff9cc')),  # P1 - Gold
    ('BACKGROUND', (0, 2), (0, 2), colors.lightgrey),  # P2 - Gray
    ('BACKGROUND', (0, 3), (0, 3), colors.lightgrey),  # N1 - Gray
    ('BACKGROUND', (0, 4), (0, 4), colors.HexColor('#cce5ff')),  # N2 - Blue
]))
story.append(abbrev_table)
story.append(Spacer(1, 0.2*inch))

# Summary Section
summary_style = ParagraphStyle('Summary', parent=styles['Heading2'], fontSize=14, textColor=colors.HexColor('#2c5aa0'))
story.append(Paragraph("Weekly Summary", summary_style))
story.append(Spacer(1, 0.1*inch))

total_bullish = len(p1_signals)
total_neutral = len(hold_signals)
total_bearish = len(n2_signals)
total = len(df_results)
pct_bullish = (total_bullish / total * 100) if total > 0 else 0

summary_data = [
    ['Signal Type', 'Count', 'Percentage'],
    [f'BUY (P1 - Gold)', str(total_bullish), f'{pct_bullish:.1f}%'],
    [f'HOLD (P2/N1 - Gray)', str(total_neutral), f'{(total_neutral/total*100):.1f}%'],
    [f'SELL (N2 - Blue)', str(total_bearish), f'{(total_bearish/total*100):.1f}%'],
    ['Total Scanned', str(total), '100%']
]

summary_table = Table(summary_data, colWidths=[3*inch, 1*inch, 1.5*inch])
summary_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 11),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ('FONTSIZE', (0, 1), (-1, -1), 10),
    ('TOPPADDING', (0, 1), (-1, -1), 6),
    ('BOTTOMPADDING', (0, 1), (-1, -1), 6),
]))
story.append(summary_table)
story.append(Spacer(1, 0.2*inch))

# Market Sentiment
if pct_bullish > 60:
    sentiment = "VERY BULLISH - Many opportunities"
elif pct_bullish > 40:
    sentiment = "BULLISH - Good opportunities"
elif pct_bullish > 20:
    sentiment = "NEUTRAL - Selective opportunities"
else:
    sentiment = "BEARISH - Few opportunities, preserve cash"

story.append(Paragraph(f"<b>Market Sentiment:</b> {sentiment}", styles['Normal']))
story.append(Spacer(1, 0.2*inch))

# Re-Optimization Alerts (if any)
if len(reopt_alerts) > 0:
    alert_style = ParagraphStyle('Alert', parent=styles['Heading3'], fontSize=12, textColor=colors.red if alert_severity == 'RED' else colors.orange, spaceAfter=6)
    story.append(Paragraph(f"🚦 Universe Health: {alert_severity}", alert_style))
    
    alert_data = [['Condition', 'Status', 'Action Required']]
    for alert in reopt_alerts:
        alert_data.append([
            alert['condition'],
            alert['severity'],
            alert['action']
        ])
    
    alert_table = Table(alert_data, colWidths=[1.5*inch, 1*inch, 3*inch])
    alert_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.red if alert_severity == 'RED' else colors.orange),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#fff3cd') if alert_severity == 'YELLOW' else colors.HexColor('#f8d7da')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ]))
    story.append(alert_table)
    story.append(Spacer(1, 0.15*inch))
    
    if alert_severity == 'RED':
        story.append(Paragraph("<b>🔴 CRITICAL ACTION:</b> Re-screen S&P 500 before next week's trading", styles['Normal']))
    elif alert_severity == 'YELLOW':
        story.append(Paragraph("<b>🟡 RECOMMENDED:</b> Plan universe re-optimization within 1-2 months", styles['Normal']))
    story.append(Spacer(1, 0.2*inch))

# Action Items
story.append(Paragraph("Action Items for Monday", summary_style))
story.append(Spacer(1, 0.1*inch))

action_items = []

# Point 1: SELL - Only show N2 positions you actually own with limit prices
if len(df_positions) > 0:
    owned_tickers = df_positions['Ticker'].tolist()
    n2_owned = n2_signals[n2_signals['Ticker'].isin(owned_tickers)]
    if len(n2_owned) > 0:
        action_items.append(f"1. ⚠️ URGENT: SELL {len(n2_owned)} N2 position(s) (9:30-10:00am)")
        for _, row in n2_owned.iterrows():
            ticker = row['Ticker']
            current_price = row['Close']
            # Get shares from portfolio
            owned_shares = df_positions[df_positions['Ticker'] == ticker]['Shares'].values[0]
            # Limit sell: 1% below current (aggressive exit)
            limit_price = current_price * 0.99
            action_items.append(f"   • {ticker}: SELL {int(owned_shares)} shares, Limit ${limit_price:.2f} (Current: ${current_price:.2f})")
    else:
        action_items.append("1. ✅ No sells required")
else:
    action_items.append("1. No positions to sell (portfolio empty)")

# Point 2: BUY - Show specific positions with limit prices and shares
if len(p1_signals) > 0:
    current_positions = len(df_positions)
    positions_to_add = min(max_positions - current_positions, len(p1_signals))
    if strategy_week == 1 and current_positions == 0:
        positions_to_add = min(3, len(p1_signals))  # Week 1: Start with 3
    
    total_deploy = position_value * positions_to_add
    top_buys = p1_signals.head(positions_to_add)
    
    action_items.append(f"2. BUY {positions_to_add} new P1 position(s) (10:00-10:30am, Total: ${total_deploy:,.0f})")
    
    # List specific positions with limit prices and shares
    for _, row in top_buys.iterrows():
        ticker = row['Ticker']
        friday_close = row['Close']
        # Calculate shares from position size (based on Friday close)
        shares = int(position_value / friday_close)
        # Limit buy: Friday close + 1.5% (balanced approach)
        limit_price = friday_close * 1.015
        actual_cost = shares * limit_price
        action_items.append(f"   • {ticker}: BUY {shares} shares, Limit ${limit_price:.2f} (Fri: ${friday_close:.2f}) = ${actual_cost:,.0f}")
else:
    action_items.append("2. No new buys available - hold cash")

# Point 3: MONITOR - Only show if you have positions
if len(df_positions) > 0:
    owned_tickers = df_positions['Ticker'].tolist()
    hold_owned = hold_signals[hold_signals['Ticker'].isin(owned_tickers)]
    if len(hold_owned) > 0:
        action_items.append(f"3. Monitor {len(hold_owned)} holding position(s) for state changes: {', '.join(hold_owned['Ticker'].tolist())}")
    else:
        action_items.append(f"3. All {len(df_positions)} position(s) are in P1 (strong) - no monitoring needed")
else:
    action_items.append("3. No positions to monitor yet")

for item in action_items:
    story.append(Paragraph(item, styles['Normal']))
    story.append(Spacer(1, 0.05*inch))

# Portfolio Holdings Section
story.append(Spacer(1, 0.2*inch))
story.append(Paragraph("Current Portfolio Holdings", summary_style))
story.append(Spacer(1, 0.1*inch))

if len(df_positions) == 0:
    story.append(Paragraph(f"<b>Status:</b> No positions - 100% CASH (${starting_cash:,.0f})", styles['Normal']))
    story.append(Spacer(1, 0.05*inch))
    if strategy_week == 1:
        story.append(Paragraph(f"<b>Week {strategy_week} Plan:</b> Start with 3 positions (${position_value:,.0f} each = 21% deployed)", styles['Normal']))
    else:
        story.append(Paragraph(f"<b>Recommendation:</b> Consider adding {min(max_positions, 3)} positions", styles['Normal']))
else:
    # Portfolio summary table
    portfolio_summary_data = [
        ['Metric', 'Value'],
        ['Active Positions', f'{len(df_positions)}/{max_positions}'],
        ['Total Cost Basis', f'${total_cost:,.0f}'],
        ['Current Value', f'${total_value:,.0f}'],
        ['Total P/L', f'${total_pl:+,.0f} ({total_pl_pct:+.1f}%)'],
        ['Cash Remaining', f'${cash_remaining:,.0f}'],
        ['Deployed / Cash', f'{deployed_pct:.1f}% / {100-deployed_pct:.1f}%']
    ]
    
    portfolio_summary_table = Table(portfolio_summary_data, colWidths=[2.5*inch, 2.5*inch])
    portfolio_summary_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (0, -1), 'LEFT'),
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('TOPPADDING', (0, 1), (-1, -1), 5),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 5),
    ]))
    story.append(portfolio_summary_table)
    story.append(Spacer(1, 0.15*inch))
    
    # Holdings detail table
    holdings_data = [['Ticker', 'Entry', 'Current', 'P/L %', 'Entry State', 'Current State', 'Signal']]
    for pos in position_summaries:
        # Format state change indicator
        state_indicator = pos['Current_State']
        if pos['State_Change']:
            state_indicator += '*'  # Mark changed states
        
        holdings_data.append([
            pos['Ticker'],
            pos['Entry_Price'],
            pos['Current_Price'],
            pos['P/L_%'],
            pos['Entry_State'],
            state_indicator,
            pos['Signal']
        ])
    
    holdings_table = Table(holdings_data, colWidths=[0.7*inch, 0.9*inch, 0.9*inch, 0.8*inch, 0.8*inch, 0.9*inch, 0.9*inch])
    holdings_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#f0f8ff')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ]))
    story.append(holdings_table)
    story.append(Spacer(1, 0.05*inch))
    story.append(Paragraph("* State changed since entry", styles['Italic']))
    
    # Alerts
    if len(n2_positions) > 0:
        story.append(Spacer(1, 0.1*inch))
        alert_style = ParagraphStyle('Alert', parent=styles['Normal'], textColor=colors.red, fontSize=10)
        story.append(Paragraph(f"<b>⚠️ URGENT:</b> {len(n2_positions)} position(s) in N2 - SELL Monday!", alert_style))

# Re-Optimization Alerts (if any)
if len(reopt_alerts) > 0:
    story.append(Spacer(1, 0.2*inch))
    alert_header_style = ParagraphStyle('AlertHeader', parent=styles['Heading3'], fontSize=12, 
                                       textColor=colors.red if alert_severity == 'RED' else colors.orange, spaceAfter=6)
    story.append(Paragraph(f"🚦 Universe Health Check: {alert_severity}", alert_header_style))
    
    alert_data = [['Condition', 'Status', 'Action Required']]
    for alert in reopt_alerts:
        alert_data.append([
            alert['condition'],
            alert['severity'],
            alert['action'][:100] + '...' if len(alert['action']) > 100 else alert['action']  # Truncate if too long
        ])
    
    alert_table = Table(alert_data, colWidths=[1.5*inch, 1*inch, 3*inch])
    alert_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.red if alert_severity == 'RED' else colors.orange),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#fff3cd') if alert_severity == 'YELLOW' else colors.HexColor('#f8d7da')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ]))
    story.append(alert_table)
    story.append(Spacer(1, 0.1*inch))
    
    if alert_severity == 'RED':
        story.append(Paragraph("<b>🔴 CRITICAL:</b> Re-screen S&P 500 before next week", styles['Normal']))
    elif alert_severity == 'YELLOW':
        story.append(Paragraph("<b>🟡 RECOMMENDED:</b> Plan re-optimization within 1-2 months", styles['Normal']))

story.append(PageBreak())

# Detailed Results by Category
story.append(Paragraph("Detailed Stock Analysis", summary_style))
story.append(Spacer(1, 0.1*inch))

# Function to create table for each category
def create_category_table(df_category, title, bg_color):
    if len(df_category) == 0:
        return None
    
    # Sort by ROC for P1, by Distance for others
    if title.startswith('BUY'):
        df_category = df_category.sort_values('ROC_4W_%', ascending=False)
    else:
        df_category = df_category.sort_values('Distance_%', ascending=False)
    
    table_data = [['Ticker', 'Price', 'D200', 'Dist %', 'ROC %', 'State']]
    for _, row in df_category.iterrows():
        table_data.append([
            row['Ticker'],
            f"${row['Close']:.2f}",
            f"${row['D200']:.2f}",
            f"{row['Distance_%']:+.1f}%",
            f"{row['ROC_4W_%']:+.1f}%",
            row['State']
        ])
    
    table = Table(table_data, colWidths=[0.8*inch, 1*inch, 1*inch, 0.9*inch, 0.9*inch, 0.7*inch])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
        ('BACKGROUND', (0, 1), (-1, -1), bg_color),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ]))
    return table

# 1. BUY Signals (P1)
story.append(Paragraph(f"1. BUY Signals - P1 (Gold) [{len(p1_signals)} stocks]", abbrev_style))
story.append(Paragraph("Action: Enter new positions or add to existing. Sorted by momentum (highest first).", styles['Normal']))
story.append(Spacer(1, 0.05*inch))
if len(p1_signals) > 0:
    buy_table = create_category_table(p1_signals, 'BUY', colors.HexColor('#fff9cc'))
    story.append(buy_table)
else:
    story.append(Paragraph("No BUY signals this week.", styles['Normal']))
story.append(Spacer(1, 0.15*inch))

# 2. HOLD Signals (P2/N1)
story.append(Paragraph(f"2. HOLD Signals - P2/N1 (Gray) [{len(hold_signals)} stocks]", abbrev_style))
story.append(Paragraph("Action: Continue holding. Monitor for state changes (upgrade to P1 or downgrade to N2).", styles['Normal']))
story.append(Spacer(1, 0.05*inch))
if len(hold_signals) > 0:
    hold_table = create_category_table(hold_signals, 'HOLD', colors.lightgrey)
    story.append(hold_table)
else:
    story.append(Paragraph("No HOLD positions.", styles['Normal']))
story.append(PageBreak())

# 3. SELL Signals (N2)
story.append(Paragraph(f"3. SELL Signals - N2 (Blue) [{len(n2_signals)} stocks]", abbrev_style))
story.append(Paragraph("Action: EXIT immediately at Monday market open. Trend is broken.", styles['Normal']))
story.append(Spacer(1, 0.05*inch))
if len(n2_signals) > 0:
    sell_table = create_category_table(n2_signals, 'SELL', colors.HexColor('#ffe5e5'))
    story.append(sell_table)
else:
    story.append(Paragraph("No SELL signals - all positions healthy!", styles['Normal']))

# Build PDF
doc.build(story)

print(f"\n✅ Results saved:")
print(f"   📊 CSV: {output_file.name}")
print(f"   📄 PDF: {pdf_file.name}")
print(f"📁 Location: {output_file.parent.absolute()}")
print(f"📂 Old scans archived to: {archive_dir.name}/")

## GHB Strategy Quick Reference

### Your Optimized Portfolio
**25 stocks:** Top-ranked S&P 500 stocks meeting GHB volatility criteria  
**Expected:** 46.80% CAGR, ~7 trades/year, 62.86% win rate (2021-2025 backtest)

### Entry Rules (BUY)
- State = P1 (Gold)
- Price > 200-day SMA
- Strong momentum (ROC > 5% OR distance > 10%)

### Hold Rules
- **P1 (Gold):** Continue holding, consider adding
- **P2 (Gray):** Hold through consolidation
- **N1 (Gray):** Hold through shallow pullback

### Exit Rules (SELL)
- State = N2 (Blue)
- Price < 200-day SMA
- Weak momentum
- **Execute Monday at open!**

### Expected Performance
- **CAGR:** 46.80% (S&P 500 optimized, 10% positions, 10 max holdings)
- **Total Return (5yr):** 586.78% ($110k → $755k)
- **Trades Per Year:** ~7
- **Win Rate:** 62.86%
- **Avg Win:** +74%
- **Avg Loss:** -12%
- **Hold Period:** 8-12 months
- **Max Drawdown:** -25.24%

### Risk Management
- Max 10% per position ($11,000 per trade)
- Up to 10 concurrent positions
- 20-30% cash reserve recommended
- Weekly monitoring only (10-15 minutes)

### Execution Guidelines
**Monday 9:30-10:00am: SELL N2 (URGENT)**
- Execute all sells first 30 minutes
- Limit: Friday close - 1% (aggressive exit)
- Don't wait for bounce - exit fast
- If missed Monday, sell Tuesday morning

**Monday 10:00-10:30am: BUY P1 (PATIENT)**
- Wait for market to settle after open
- Limit: Friday close + 1.5% (balanced entry)
- If limit doesn't fill, WAIT for next Friday
- Don't chase mid-week (maintain weekly discipline)

**Key Principle:** In a hurry to stop losses, patient to enter winners

---
**Next Steps:**
1. Review signals above
2. Execute SELLS first (9:30-10:00am Monday)
3. Execute BUYS second (10:00-10:30am Monday)
4. Update portfolio CSV Monday evening
5. Run this notebook again next Friday

**Documentation:** See `docs/GHB_STRATEGY_GUIDE.md` for complete strategy details  
**Portfolio List:** See `data/ghb_optimized_portfolio.txt` for your 25 S&P 500 optimized stocks  
**Backtest Report:** See `docs/BACKTEST_ANALYSIS_REPORT.md` for complete performance analysis